In [1]:
import pandas as pd
from datasets import load_dataset
from collections import OrderedDict

# List of language codes for the desired languages
language_codes = {
    "en": "English",
    "es": "Spanish",
    "ru": "Russian",
    "ja": "Japanese",
    "zh-CN": "Mandarin",
}

transcripts = []

for lang_code, lang_name in language_codes.items():
    try:
        dataset = load_dataset("mozilla-foundation/common_voice_11_0", lang_code, split="test", streaming=True)
        for x in dataset:
            transcripts.append({"language": lang_name, "sentence": x["sentence"], "audio": x["audio"]})
        print(f"Loaded transcripts for language: {lang_name}")
    except Exception as e:
        print(f"Error loading transcripts for language: {lang_name}. Error: {str(e)}")

df = pd.DataFrame(transcripts)
print(df)

Reading metadata...: 16354it [00:00, 56570.03it/s]


In [8]:
print(len(transcripts['Russian']))

9630
